In [13]:
import psycopg2
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point,Polygon
from datetime import date

In [14]:
db_con = {
    'host':'localhost', 
    'database':'postgres',
    'user':'postgres', 
    'password':'root'
}

In [15]:
query = '''
SELECT 
    opr.opr_fim_plantio as fim_plantio,
    opr.opr_fim_colheita as fim_colheita,
    glb.glb_id as id,
    glb.glb_poligono as gleba
FROM opr_operacao AS opr
inner join 
	glb_gleba as glb
on glb.opr_id = opr.opr_id
and glb.glb_id in (1633,100,27)
'''

In [16]:
with psycopg2.connect(**db_con) as conn:
    df = gpd.read_postgis(query,conn, geom_col='gleba')

/home/pedro/.local/lib/python3.10/site-packages/geopandas/io/sql.py:170: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [17]:
df['centroid'] = df['gleba'].centroid
df['latitude'] = df['centroid'].y
df['longitude'] = df['centroid'].x

/tmp/ipykernel_4875/1757581452.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['centroid'] = df['gleba'].centroid


In [18]:
df['fim_plantio'] = pd.to_datetime(df['fim_plantio'])
df['fim_plantio_epoch'] = df['fim_plantio'].astype(int) // 10**9
df['fim_colheita'] = pd.to_datetime(df['fim_colheita'])
df['fim_colheita_epoch'] = df['fim_colheita'].astype(int) // 10**9

In [19]:
teste = pd.Series(pd.to_datetime('2018-10-30'))
print(teste.astype(int) // 10**9)

0    1540857600
dtype: int64


In [20]:
df.dtypes

fim_plantio           datetime64[ns]
fim_colheita          datetime64[ns]
id                             int64
gleba                       geometry
centroid                    geometry
latitude                     float64
longitude                    float64
fim_plantio_epoch              int64
fim_colheita_epoch             int64
dtype: object

In [21]:
df.head(10)

,fim_plantio,fim_colheita,id,gleba,centroid,latitude,longitude,fim_plantio_epoch,fim_colheita_epoch
0,2018-10-20,2019-02-20,27,"POLYGON ((-52.57115 -23.46758, -52.56683 -23.4...",POINT (-52.56964 -23.46209),-23.462091,-52.569642,1539993600,1550620800
1,2018-10-27,2019-02-28,100,"POLYGON ((-52.61886 -23.50106, -52.62050 -23.5...",POINT (-52.61885 -23.50001),-23.500009,-52.618849,1540598400,1551312000
2,2018-10-20,2019-02-28,1633,"POLYGON ((-49.78642 -22.68941, -49.78542 -22.6...",POINT (-49.78018 -22.68878),-22.688777,-49.780181,1539993600,1551312000


In [22]:
df['fim_plantio'] = pd.to_datetime(df['fim_plantio'], format='%Y-%m-%d').astype(str)
df['fim_colheita'] = pd.to_datetime(df['fim_colheita'], format='%Y-%m-%d').astype(str)
df['gleba'] = df['gleba'].apply(lambda geom: ','.join([f'[{x},{y}]' for x, y in geom.exterior.coords]))
df.dtypes

/tmp/ipykernel_4875/2256651839.py:3: UserWarning: Geometry column does not contain geometry.
  df['gleba'] = df['gleba'].apply(lambda geom: ','.join([f'[{x},{y}]' for x, y in geom.exterior.coords]))


fim_plantio             object
fim_colheita            object
id                       int64
gleba                   object
centroid              geometry
latitude               float64
longitude              float64
fim_plantio_epoch        int64
fim_colheita_epoch       int64
dtype: object

In [23]:
for row in df.iterrows():
    print(row[1]['id'])
    url = f'https://indices-svsux4mgsa-uc.a.run.app/time_series?dt_antes={row[1]["fim_plantio"]}&dt_depois={row[1]["fim_colheita"]}&safra=1&filter=%22whittaker%22&geom=[[{row[1]["gleba"]}]]'
    print(url)

27
https://indices-svsux4mgsa-uc.a.run.app/time_series?dt_antes=2018-10-20&dt_depois=2019-02-20&safra=1&filter=%22whittaker%22&geom=[[[-52.571149,-23.467578],[-52.566832,-23.460926],[-52.566836,-23.460606],[-52.566873,-23.460442],[-52.566643,-23.460208],[-52.566501,-23.460056],[-52.566339,-23.460092],[-52.56588,-23.459437],[-52.565422,-23.458627],[-52.565582,-23.458198],[-52.565861,-23.457928],[-52.565818,-23.457586],[-52.566169,-23.45701],[-52.573432,-23.462384],[-52.572141,-23.465653],[-52.571625,-23.467043],[-52.571149,-23.467578]]]
100
https://indices-svsux4mgsa-uc.a.run.app/time_series?dt_antes=2018-10-27&dt_depois=2019-02-28&safra=1&filter=%22whittaker%22&geom=[[[-52.618858,-23.501058],[-52.620499,-23.502401],[-52.620982,-23.501889],[-52.621143,-23.501801],[-52.621368,-23.501574],[-52.621561,-23.501348],[-52.621679,-23.501141],[-52.621744,-23.500895],[-52.619555,-23.499346],[-52.617227,-23.497737],[-52.616519,-23.498327],[-52.61609,-23.498721],[-52.615693,-23.499065],[-52.616519,

In [27]:
import requests
import json 

for row in df.iterrows():
    print(row[1]['id'])
    url = f'https://history.openweathermap.org/data/2.5/history/city?lat={row[1]["latitude"]}&lon={row[1]["longitude"]}&units=metric&appid=d1955cd784b7f8da530f2f7ec6e0d652&type=weekly'
    print(url)
    res = requests.get(url)
    if res.status_code == 200:
        with open(f'../open_weather/{row[1]["id"]}.json','w') as file:
            file.write(json.dumps(res.json()))

27
https://history.openweathermap.org/data/2.5/history/city?lat=-23.46209111402962&lon=-52.56964206024581&units=metric&appid=d1955cd784b7f8da530f2f7ec6e0d652&type=weekly
100
https://history.openweathermap.org/data/2.5/history/city?lat=-23.500009295950996&lon=-52.618849190658715&units=metric&appid=d1955cd784b7f8da530f2f7ec6e0d652&type=weekly
1633
https://history.openweathermap.org/data/2.5/history/city?lat=-22.688776968524706&lon=-49.78018056845981&units=metric&appid=d1955cd784b7f8da530f2f7ec6e0d652&type=weekly
